In [3]:
!pip install torch transformers
!pip install -U datasets
!git clone https://github.com/davidjurgens/citation-function

fatal: destination path 'citation-function' already exists and is not an empty directory.


In [1]:
import os
with open('api_openrouter.txt', 'r') as f:
  API=f.read()


In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset
import numpy as np
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive')
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]
academic_personas = [
    "PhD Student",
    "Postdoctoral Researcher",
    "Assistant Professor",
    "Associate Professor",
    "Full Professor",
    "Researcher in Industry",
]
disciplines = [
        "Artificial Intelligence",
        "Machine Learning",
        "Computer Vision",
        "Natural Language Processing",
        "Cybersecurity",
        "Distributed Systems",
        "Algorithms and Complexity",
        "Databases",
        "Human-Computer Interaction",
        "Quantum Computing",
        "Embedded Systems",
        "Software Engineering",
        "Classical Mechanics",
        "Quantum Mechanics",
        "Relativity",
        "Condensed Matter Physics",
        "Particle Physics",
        "Nuclear Physics",
        "Optics and Photonics",
        "Statistical Mechanics",
        "Astrophysics",
        "Acoustics",
        "Plasma Physics",
        "Biophysics",
        "Algebra",
        "Real Analysis",
        "Complex Analysis",
        "Topology",
        "Differential Geometry",
        "Number Theory",
        "Probability Theory",
        "Statistics",
        "Numerical Analysis",
        "Optimization",
        "Mathematical Logic",
        "Applied Mathematics",
        "Organic Chemistry",
        "Inorganic Chemistry",
        "Physical Chemistry",
        "Analytical Chemistry",
        "Biochemistry",
        "Materials Chemistry",
        "Theoretical and Computational Chemistry",
        "Environmental Chemistry",
        "Polymer Chemistry",
        "Surface Chemistry",
        "Molecular Biology",
        "Cell Biology",
        "Genetics and Genomics",
        "Microbiology",
        "Neurobiology",
        "Ecology",
        "Evolutionary Biology",
        "Developmental Biology",
        "Physiology",
        "Botany",
        "Zoology",
        "Systems Biology",
        "Internal Medicine",
        "Surgery",
        "Pediatrics",
        "Psychiatry",
        "Cardiology",
        "Oncology",
        "Neurology",
        "Radiology",
        "Anesthesiology",
        "Emergency Medicine",
        "Primary Care",
        "Public Health and Epidemiology",
        "Mechanical Engineering",
        "Electrical Engineering",
        "Civil Engineering",
        "Chemical Engineering",
        "Aerospace Engineering",
        "Biomedical Engineering",
        "Environmental Engineering",
        "Materials Engineering",
        "Industrial Engineering",
        "Computer Engineering",
        "Robotics",
        "Systems Engineering",
        "Microeconomics",
        "Macroeconomics",
        "Econometrics",
        "Behavioral Economics",
        "Development Economics",
        "International Economics",
        "Public Economics",
        "Environmental Economics",
        "Health Economics",
        "Labor Economics",
        "Financial Economics",
        "Cognitive Psychology",
        "Developmental Psychology",
        "Social Psychology",
        "Clinical Psychology",
        "Neuropsychology",
        "Industrial-Organizational Psychology",
        "Health Psychology",
        "Personality Psychology",
        "Psychometrics",
        "Educational Psychology",
        "Sociological Theory",
        "Social Stratification",
        "Cultural Sociology",
        "Sociology of Education",
        "Urban Sociology",
        "Medical Sociology",
        "Economic Sociology",
        "Political Sociology",
        "Family Sociology",
        "Quantitative Methods",
        "Qualitative Methods",
        "Mineralogy and Petrology",
        "Structural Geology",
        "Sedimentology",
        "Stratigraphy",
        "Paleontology",
        "Geomorphology",
        "Geochemistry",
        "Geophysics",
        "Hydrogeology",
        "Environmental Geology",
        "Remote Sensing in Geology",
        "Observational Astronomy",
        "Theoretical Astronomy",
        "Planetary Science",
        "Stellar Astrophysics",
        "Extragalactic Astronomy",
        "Cosmology",
        "Astrochemistry",
        "High-energy Astrophysics",
        "Radio Astronomy",
        "Infrared and Optical Astronomy",
        "Climate Science",
        "Conservation Biology",
        "Environmental Chemistry",
        "Ecology",
        "Environmental Policy and Management",
        "Hydrology",
        "Soil Science",
        "Atmospheric Science",
        "Sustainability Science",
        "Environmental Impact Assessment",
        "Comparative Politics",
        "International Relations",
        "Political Theory",
        "Public Policy",
        "Political Economy",
        "Electoral Studies",
        "Political Behavior",
        "Public Administration",
        "Security Studies",
        "Governance and Institutions",
        "Ancient History",
        "Medieval History",
        "Early Modern History",
        "Modern History",
        "Economic History",
        "Social and Cultural History",
        "Military History",
        "History of Science and Technology",
        "Oral History",
        "Public History",
        "Phonetics",
        "Phonology",
        "Morphology",
        "Syntax",
        "Semantics",
        "Pragmatics",
        "Sociolinguistics",
        "Psycholinguistics",
        "Computational Linguistics",
        "Historical Linguistics",
        "Field Linguistics",
        "Cultural Anthropology",
        "Biological Anthropology",
        "Archaeology",
        "Linguistic Anthropology",
        "Medical Anthropology",
        "Economic Anthropology",
        "Urban Anthropology",
        "Ethnography",
        "Visual Anthropology",
        "Anthropology of Religion",
        "Metaphysics",
        "Epistemology",
        "Ethics",
        "Political Philosophy",
        "Philosophy of Mind",
        "Philosophy of Science",
        "Logic",
        "Aesthetics",
        "Philosophy of Language",
        "History of Philosophy",
        "Physical Geography",
        "Human Geography",
        "Geographic Information Systems (GIS)",
        "Urban Geography",
        "Economic Geography",
        "Political Geography",
        "Cultural Geography",
        "Environmental Geography",
        "Cartography",
        "Remote Sensing",
        "Ancient Art",
        "Medieval Art",
        "Renaissance Art",
        "Baroque and Rococo",
        "Modern Art",
        "Contemporary Art",
        "Non-Western Art Histories",
        "Iconography",
        "Museum Studies",
        "Conservation and Restoration",
        "Harmony",
        "Counterpoint",
        "Form and Analysis",
        "Aural Skills",
        "Tonal Theory",
        "Atonal and Serial Techniques",
        "Ethnomusicology",
        "Music Cognition",
        "Music Notation and Editorial Practices",
        "Contemporary Music Theory",
        "Comparative Literature",
        "Literary Theory and Criticism",
        "Medieval and Early Modern Literature",
        "Modern and Contemporary Literature",
        "Postcolonial Literature",
        "Genre Studies",
        "Narrative Theory",
        "Digital Humanities and Literature",
        "Translation Studies",
        "Children's and Young Adult Literature",
        "Curriculum and Instruction",
        "Educational Psychology",
        "Assessment and Evaluation",
        "Educational Policy",
        "Special Education",
        "Early Childhood Education",
        "Higher Education Studies",
        "Technology in Education",
        "Adult and Continuing Education",
        "Multicultural Education"
    ]


disciplines_1 = [
    "Computer Science",
    "Physics",
    "Mathematics",
    "Chemistry",
    "Biology",
    "Medicine",
    "Engineering",
    "Economics",
    "Psychology",
    "Sociology",
    "Geology",
    "Astronomy",
    "Environmental Science",
    "Political Science",
    "History",
    "Linguistics",
    "Anthropology",
    "Philosophy",
    "Geography",
    "Art History",
    "Music Theory",
    "Literature",
    "Education"
]
place_sentence=['beginning', 'middle', 'end']
citation_intent=['BACKGROUND','USES' ,'COMPARES OR CONTRASTS' ,'MOTIVATION', 'CONTINUATION', 'FUTURE' ]
prompt_intent={
    "BACKGROUND":"to provide relevant information for this domain (BACKGROUND).",
    "MOTIVATION":"to illustrate need for data, goals, methods, etc.(MOTIVATION)",
    "USES":" to use data, methods, etc from the citation. (USES) ",  
    "CONTINUATION":"to extend the citation's data, methods, etc. (CONTINUATION)",
    "COMPARES OR CONTRASTS":"to express similarity/differences to the citation (COMPARES OR CONTRASTS)",
    "FUTURE": "to be a potential avenue for future work (FUTURE)"
}
citation_dict={
    0:"BACKGROUND",
    1:"USES",
    2:"COMPARES OR CONTRASTS",
    3:"MOTIVATION",
    4:"CONTINUATION",
    5:"FUTURE"
}
import random

def generate_researcher_prompt(numero_classe,academic_personas=academic_personas, disciplines=disciplines, citation_intent=citation_intent,prompt_intent=prompt_intent):
  persona = random.choice(academic_personas)
  discipline = random.choice(disciplines)
  intent=citation_dict[numero_classe]
  intent=prompt_intent[intent]
  prompt = f"Imagine you are a {persona} specializing in {discipline}. Your task is to write 10 citations whose goal are {intent}, the said citation should be written as @@CITATION and be in the {place_sentence[random.randint(0,2)]} of the sentence"
  return prompt

# Example usage:
print(generate_researcher_prompt(numero_classe=0))
print(generate_researcher_prompt(numero_classe=1))
print(generate_researcher_prompt(numero_classe=2))
print(generate_researcher_prompt(numero_classe=3))
print(generate_researcher_prompt(numero_classe=4))
print(generate_researcher_prompt(numero_classe=5))



c:\Users\chrif\Downloads\synthdata1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imagine you are a Postdoctoral Researcher specializing in Political Sociology. Your task is to write 10 citations whose goal are to provide relevant information for this domain (BACKGROUND)., the said citation should be written as @@CITATION and be in the middle of the sentence
Imagine you are a Researcher in Industry specializing in Stellar Astrophysics. Your task is to write 10 citations whose goal are  to use data, methods, etc from the citation. (USES) , the said citation should be written as @@CITATION and be in the middle of the sentence
Imagine you are a PhD Student specializing in Electoral Studies. Your task is to write 10 citations whose goal are to express similarity/differences to the citation (COMPARES OR CONTRASTS), the said citation should be written as @@CITATION and be in the middle of the sentence
Imagine you are a Assistant Professor specializing in Human Geography. Your task is to write 10 citations whose goal are to illustrate need for data, goals, methods, etc.(MO

In [3]:
def id_to_string(id):
  try:
    with open('/content/citation-function/data/adjudicated-and-supplemental/'+id+'-parscit.130908.xml.txt', 'r') as f:
      file_contents = f.read()
  except FileNotFoundError:
    with open('/content/citation-function/data/adjudicated-and-supplemental/'+id+'.xml.txt', 'r') as f:
      file_contents = f.read()


  return file_contents

def string_few_shots(nb_of_citations):
  output_string = ""
  for j in range(0,6): #pour chaque classe
    compteur=nb_of_citations
    for count,letuple in enumerate (zip(train['intent'],train['cleaned_cite_text'],train['section_name'])): #on parcourt toutes les citations
      if(letuple[0] == j): #si elle a la bonne classe
        #output_string += f"CLASS: {citation_dict[j]}  SECTION NAME:{letuple[2]} CITATION: {letuple[1]}\n" #on l'ajoute à la string
        output_string += f"CLASS: {citation_dict[j]} CITATION: {letuple[1]}\n" #on l'ajoute à la string

        compteur-=1 # et on décrémente
        if(compteur==0): #si on arrive à zéro on passe à la classe d'après
          break
  return output_string



def string_few_shots_2(nb_of_citations,class_number):
  output_string = ""
  j=class_number
  compteur=nb_of_citations
  for count,letuple in enumerate (zip(train['intent'],train['cleaned_cite_text'],train['section_name'])): #on parcourt toutes les citations
    if(letuple[0] == j): #si elle a la bonne classe
      #output_string += f"CLASS: {citation_dict[j]}  SECTION NAME:{letuple[2]} CITATION: {letuple[1]}\n" #on l'ajoute à la string
      output_string += f"CLASS: {citation_dict[j]} CITATION: {letuple[1]}\n" #on l'ajoute à la string

      compteur-=1 # et on décrémente
      if(compteur==0): #si on arrive à zéro on passe à la classe d'après
        break
  return output_string


In [4]:
PROMPT_2=(generate_researcher_prompt(0)
+'\n\nhere are a few examples of said class:\n\n'+ string_few_shots_2(1,0)+ '\n\n')
print(PROMPT_2)

Imagine you are a Assistant Professor specializing in Theoretical Astronomy. Your task is to write 10 citations whose goal are to provide relevant information for this domain (BACKGROUND)., the said citation should be written as @@CITATION and be in the end of the sentence

here are a few examples of said class:

CLASS: BACKGROUND CITATION: Thus , over the past few years , along with advances in the use of learning and statistical methods for acquisition of full parsers ( @@CITATION ; Charniak , 1997a ; Charniak , 1997b ; Ratnaparkhi , 1997 ) , significant progress has been made on the use of statistical learning methods to recognize shallow parsing patterns syntactic phrases or words that participate in a syntactic relationship ( Church , 1988 ; Ramshaw and Marcus , 1995 ; Argamon et al. , 1998 ; Cardie and Pierce , 1998 ; Munoz et al. , 1999 ; Punyakanok and Roth , 2001 ; Buchholz et al. , 1999 ; Tjong Kim Sang and Buchholz , 2000 ) .





In [6]:
PROMPT=(generate_researcher_prompt(1)
      +'\n\nhere are a few examples of every class:\n\n'+ string_few_shots(1)+ '\n\n')
print(PROMPT)

Imagine you are a PhD Student specializing in Political Geography. Your task is to write 10 citations whose goal are  to use data, methods, etc from the citation. (USES) , the said citation should be written as @@CITATION and be in the beginning of the sentence

here are a few examples of every class:

CLASS: BACKGROUND CITATION: Thus , over the past few years , along with advances in the use of learning and statistical methods for acquisition of full parsers ( @@CITATION ; Charniak , 1997a ; Charniak , 1997b ; Ratnaparkhi , 1997 ) , significant progress has been made on the use of statistical learning methods to recognize shallow parsing patterns syntactic phrases or words that participate in a syntactic relationship ( Church , 1988 ; Ramshaw and Marcus , 1995 ; Argamon et al. , 1998 ; Cardie and Pierce , 1998 ; Munoz et al. , 1999 ; Punyakanok and Roth , 2001 ; Buchholz et al. , 1999 ; Tjong Kim Sang and Buchholz , 2000 ) .
CLASS: USES CITATION: This was done by MERT optimization ( @

In [7]:
import requests
import json
import os

les_rez=[]
dico_rez={0:[],1:[],2:[],3:[],4:[],5:[]}

url = "https://openrouter.ai/api/v1/chat/completions"


headers = {
  "Authorization": f"Bearer {API}",
  "Content-Type": "application/json"
}

for j in range(0,6):

  
  print('NOUVELLE CLASSE:',str(j))

  for i in range(0,10):
    PROMPT=(generate_researcher_prompt(j)
      +'\n\nhere are a few examples of that class:\n\n'+ string_few_shots_2(5,j)+ '\n\n')

    print(i)
    try:
      payload = {
          "model": "deepseek/deepseek-chat-v3.1:free",
        "messages": [
          {
            "role": "user",
            "content": PROMPT
          }
        ]
          ,
      #   "provider": {
      #     "allow_fallbacks": True,
      #     "quantizations": [
      #       "fp8"
      #     ]
      # },
          "reasoning": {

        "enabled": True  # Use high reasoning effort

    }
      }
      response = requests.post(url, headers=headers, json=payload)
      les_rez.append(response.json())
      dico_rez[j].append(response.json())
    except:
      print("error")
      continue
    print(response.json())

NOUVELLE CLASSE: 0
0
{'id': 'gen-1758487456-SwfESyvaD0piczSzsA2C', 'provider': 'DeepInfra', 'model': 'deepseek/deepseek-chat-v3.1:free', 'object': 'chat.completion', 'created': 1758487456, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The study of Renaissance art is fundamentally rooted in the socio-political context of the Italian city-states, where patronage systems fueled artistic innovation (@@CITATION).\n\nGiorgio Vasari\'s seminal text, *The Lives of the Artists*, established a critical framework for understanding the period\'s artistic development and remains a primary source for art historians (@@CITATION).\n\nThe intellectual shift towards humanism, which placed renewed emphasis on classical antiquity and the individual, provided the philosophical foundation for Renaissance artistic expression (@@CITATION).\n\nThe development of linear perspective, systematically articulated by F

In [8]:
for i in range(0,6):
    for elt in dico_rez[i]:
        with open(f'DATASET3/results_openrouter_intent_{i}.txt', 'a',encoding='utf-8') as f:
            f.write(elt['choices'][0]['message']['content'] + '\n\n')